# Activity propagation along potentiated ridge

This notebook serves as a control panel for simulations of network activity propagating along a ridge of CA3 PCs receiving EC input through potentiated EC->PC synapses.

In [2]:
from datetime import datetime
import importlib
import time

from db import make_session, d_models

import PARAMETERS as P

In [3]:
SEARCH_CONFIG_ROOT = 'search_config.ridge'

### Define objective function

In [1]:
def obj(params):
    """
    Run a trial given dict of param values and return dict of results.
    
    :param params: dict of params, which in this case must include:
        RHO_PC
        
        Z_PC
        L_PC
        W_A_PC_PC
        
        P_INH_PC
        W_A_INH_PC
        
        P_PC_INH
        W_G_PC_INH
        
        W_A_PC_EC_I
        RATE_EC
        
    :return: dict of measured vals for:
        PROPAGATION
        ACTIVITY
        SPEED
    """
    return {'PROPAGATION': 0, 'ACTIVITY': None, 'SPEED': None}

### Auxiliary functions

In [6]:
def p_from_trial(trial):
    """Extract param dict from trial instance."""
    
    return {
        'RIDGE_H': trial.ridge_h,
        'RIDGE_W': trial.ridge_w,
        
        'RHO_PC': trial.rho_pc,
        
        'Z_PC': trial.z_pc,
        'L_PC': trial.l_pc,
        'W_A_PC_PC': trial.w_a_pc_pc,
        
        'P_A_INH_PC': trial.p_a_inh_pc,
        'W_A_INH_PC': trial.w_a_inh_pc,
        
        'P_G_PC_INH': trial.p_g_pc_inh,
        'W_G_PC_INH': trial.w_g_pc_inh,
        
        'W_N_PC_EC_I': trial.w_n_pc_ec_i,
        'W_RATE_EC': trial.rate_ec,
    }


def rslt_from_trial(trial):
    """Extract result dict from trial instance."""
    
    return {
        'PROPAGATION': trial.propagation,
        'ACTIVITY': trial.activity,
        'SPEED': trial.speed,
    }


def sample_x_rand(cfg):
    """Sample random x."""
    x = np.nan * np.ones(len(cfg.P_RANGES))
    
    # loop over all parameters
    for ctr, (_, p_range) in enumerate(cfg.P_RANGES):
        
        # fixed or random
        if len(p_range) == 1:  # fixed
            x[ctr] = 0
            
        else:  # random
            scale = p_range[-1]
            
            # keep sampling until valid param found
            while True:
                x_ = np.random.normal(0, scale*cfg.SGM_RAND)
                
                if -scale/2 <= x_ < scale/2:
                    break
                    
            x[ctr] = x_
            
    return x


def sample_x_prev(cfg, session, p_to_x):
    """Sample previously visited x."""
    
    # get past trials
    trials = session.query(d_models.RidgeTrial).filter_by(sim_id=cfg.SIM_ID)
    
    # get param dicts
    ps = [p_from_trial(trial) for trial in trials]
    
    # get results
    rslts = [rslt_from_trial(trial) for trial in trials]
    temp = [(rslt['PROPAGATION'], rslt['ACTIVITY'], rslt['SPEED']) for rslt in rslts]
    
    y, k, s = np.array(temp).T
    
    # get distances to targets
    d_k = np.exp(-np.abs(k - cfg.K_TARG)/cfg.ETA_K)
    d_s = np.exp(-np.abs(s - cfg.S_TARG)/cfg.ETA_S)
    
    # calculate weights
    temp = cfg.B_PREV_Y*y + cfg.B_PREV_K*d_k + cfg.B_PREV_S*d_s
    w = np.exp(cfg.A_PREV * temp / cfg.B_PREV_SUM)
    w /= w.sum()
    
    # sample idx of previous location
    idx = np.random.choice(len(ps), p=w)
    
    # get params and x
    p = ps[idx]
    
    return p_to_x(p)


def sample_x_step(cfg, session, searcher, x_prev, since_jump, p_to_x):
    """Sample new x using stepping algorithm."""
    
    n = min(cfg.N_PHI, since_jump)
    
    # sample step size
    l = np.random.exponential(cfg.L_STEP)
    
    if n == 0:
        phi_mean = np.zeros(len(x_prev))
    
    else:
        # get past n trials with this searcher id
        trials = session.query(d_models.RidgeTrial).\
            filter_by(searcher_id=searcher.id).\
            order_by(d_model.RidgeTrial.id.desc()).limit(n).all()
            
        ## get params and measurables for past results
        ps = [p_from_trial(trial) for trial in trials]
    
        # get results
        rslts = [rslt_from_trial(trial) for trial in trials]
        temp = [(rslt['PROPAGATION'], rslt['ACTIVITY'], rslt['SPEED']) for rslt in rslts]

        y, k, s = np.array(temp).T
        
        # compute optimal direction

        ## get distances to targets
        d_k = np.exp(-np.abs(k - cfg.K_TARG)/cfg.ETA_K)
        d_s = np.exp(-np.abs(s - cfg.S_TARG)/cfg.ETA_S)

        ## calculate weights
        temp = cfg.B_PHI_Y*y + cfg.B_PHI_K*d_k + cfg.B_PHI_S*d_s
        w = np.exp(cfg.A_PHI * temp / cfg.B_PHI_SUM)
        w /= w.sum()

        ## get xs for past results and dists to center of mass
        xs = np.array([p_to_x(p) for p in ps])
        dxs = xs - xs.mean(0)

        ## take weighted sum of dxs
        dx_sum = dxs.T.dot(w)

        ## get optimal direction
        phi_best = dx_sum / np.linalg.norm(dx_sum)
        phi_mean = cfg.L_PHI*phi_best
    
    # sample final step direction
    phi_ = np.random.normal(phi_mean)
    phi = phi_/np.linalg.norm(phi_)
    
    # return final x
    return x_prev + l*phi
    

def make_param_conversions(p_ranges):
    """
    Return functions for converting from p to x and x to p.
    
    :param p_ranges: list of two-element tuples; first element is param name,
        second element is range, which can either be single-element list
        specifying fixed value, or three-element list giving lower bound, upper
        bound, and scale (approx. resolution).
    """
    
    def p_to_x(p):
        """Convert param dict to x."""
        
        x = np.nan * np.zeros(len(p))
        
        for ctr, (name, p_range) in enumerate(p_ranges):
            
            if len(p_range) == 1:  # fixed val
                x[ctr] = 0
                
            else:
                # compute x from p and param range
                lb, ub, scale = p_range
                x[ctr] = scale * (p[name] - ((ub + lb)/2)) / (ub - lb)
                
        return x
    
    def x_to_p(x):
        """Convert x to param dict."""
        
        p = {}
        
        for ctr, (name, p_range) in enumerate(p_ranges):
            
            if len(p_range) == 1:  # fixed val
                p[name] = p_range[0]
                
            else:
                # compute p from x and param range
                lb, ub, scale = p_range
                p[name] = (x[ctr] * (ub - lb) / scale) + ((lb + ub) / 2)
                
        return p
    
    return p_to_x, x_to_p


def save_ridge_trial(session, searcher, seed, p, rslt):
    
    trial = d_models.RidgeTrial(
        searcher_id=searcher.id,
        seed=seed,
        
        ridge_h=p['RIDGE_H'],
        ridge_w=p['RIDGE_W'],
        
        rho_pc=p['RHO_PC'],
        
        z_pc=p['Z_PC'],
        l_pc=p['L_PC'],
        w_a_pc_pc=p['W_A_PC_PC'],
        
        p_a_inh_pc=p['P_A_INH_PC'],
        w_a_inh_pc=p['W_A_INH_PC'],
        
        p_g_pc_inh=p['P_G_PC_INH'],
        w_g_pc_inh=p['W_G_PC_INH'],
        
        w_n_pc_ec_i=p['W_N_PC_EC_I'],
        rate_ec=p['RATE_EC']
        
        propagation=rslt['PROPAGATION'],
        activity=rslt['ACTIVITY'],
        speed=rslt['SPEED'],
    )
    
    session.add(trial)
    session.commit()

### Search function

In [ ]:
def _launch_ridge_searcher(role, max_iter=10):
    """
    Launch instance of searcher exploring potentiated ridge trials.
    
    :param role: searcher role, which should correspond to search config file
    """
    # import initial config
    cfg = importlib.import_module('.'.join([SEARCH_CONFIG_ROOT, role]))
    
    # define parameter conversions
    p_to_x, x_to_p = make_param_conversions(cfg.P_RANGES)

    # connect to db
    session = make_session()
    
    # make new searcher
    searcher = d_models.RidgeSearcher(
        sim_id=cfg.SIM_ID,
        last_active=datetime.now()
        last_error=None)
    session.add(searcher)
    session.commit()
    
    since_jump = 0
    
    # loop over search iterations
    for ctr in range(max_iter):
        
        # reload search config
        importlib.reload(cfg)
        
        # attempt iteration
        try:
            
            # "additional step required" flag
            step = True
            
            # jump or stay
            if np.random.rand() < cfg.Q_JUMP:  # jump
                
                # new point or prev
                if np.random.rand() < cfg.Q_NEW:  # new
                    
                    # get next candidate x
                    x_cand = sample_x_rand(cfg)
                    x_prev = x_cand
                    step = False
                    
                else:  # prev
                    x_prev = sample_x_prev(cfg, session, p_to_x)
                
                since_jump = 0
                    
            else:  # stay
                x_prev = x.copy()
                since_jump += 1
            
            # take new step if necessary
            if step:
                x_cand = sample_x_step(cfg, session, searcher, x_prev, since_jump, p_to_x)
                move_to.append(x_cand)
            
            # ensure x_cand is within bounds
            x_cand = correct_if_out_of_bounds(x_cand, cfg)
            
            # convert x_cand to param dict and store result
            p_cand = x_to_p(x_cand)
            rslt = obj(p_cand)
            save_ridge_trial(session, searcher, seed, p_cand, rslt)
            
            # move to x_cand if propagation occurred else x_prev
            if rslt['PROPAGATION']:
                x = x_cand.copy()
            else:
                x = x_prev.copy()
            
        except:
            raise Exception('Search step failed.')
            time.sleep(5)

# Searcher control panel